# 利用CNN实现人脸关键点提取

人脸关键点(也称为人脸特征点)指的是下面的图像中,每个人脸上显示的洋红色的小点.在每个训练和测试图像中, 有一个人脸和**68**个关键点. 这些关键点标记了人脸的重要区域: 眼睛,嘴角,鼻子等. 这些关键点与许多应用相关,如人脸滤波,情感识别,姿势识别等. 在这里, 它们是编号的, 你可以看到特定范围的点与该人脸的不同部分相匹配.

![face](images/key_pts_example.png)

这组图像数据是从[YouTube 人脸数据集](https://www.cs.tau.ac.il/~wolf/ytfaces/)中提取的, 其中包含YouTube视频中的人物视频. 这些视频通过一些处理步骤进行输入, 并转换为包含一个人脸和相关关键点的图像帧集.

该人脸关键点数据集由5770张彩色图像组成, 所有这些图像都被分成训练数据集与测试数据集.

* 这些图像中有3462张个是训练图像, 供模型进行训练.
* 另外2308张是测试图像, 用于测试模型的准确性.

我把数据集的读取代码全部封装在`facial_keypoints_dataset.py`中了. 这里我们只需要调用其中的API便可以观察数据集了.

In [ ]:
import facial_keypoints_dataset as dataset
import facial_keypoints_train as train
import facial_keypoints_model_v2 as fk_model
%matplotlib inline

数据集的读取使用类完成. 其中训练集和测试集是分开的.

如果磁盘上没有数据集, 则在创建数据集对象的时候会自动下载解压, 默认会把数据下载到`data/`目录下.

In [ ]:
train_data = dataset.TrainDataset()

数据集对象是由很多样本构成的. 每个样本是一个dict, 有两个元素:

- key='image', 原始人脸图像, ndarray, shape=(w, h,c).
- key='kps', 人脸关键点, ndarray, shape=(68, 2), 其中68表示每张人脸有68个关键点, 2表示坐标.

下面我们查看其中一个样本, 并通过`show_kps()`调用把关键点绘制在人脸图像上.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# 随便选一个样本
sample = train_data[899]

image = sample['image']
kps = sample['kps']

print('image shape =', image.shape)
print('key points shape =', kps.shape)

dataset.show_kps(image, kps)

如果你多测试几组图像, 会发现每张图像的尺寸是不一样的.

所以训练数据还需要进行一些额外的预处理. 这里提供一个`train_transform()`函数专门对样本进行预处理, 它对图像和关键点依次进行如下操作:

- 缩放为固定尺寸
- 随机裁剪
- 转换为灰度图像
- 像素值归一到\[0,1\]之间
- 转换为torch tensor, 注意和numpy不同, torch表示图像是(c, h, w), 这里需要进行转换

我们不需要手动对每个样本调用这个函数, 只需要在创建数据集对象时把这个函数传入即可.

经过这样的处理, 所有图像都会被转换到\[1, 96, 96\]这样的维度.

In [ ]:
# 重新创建一个数据集对象, 这里进行预处理操作
train_data = dataset.TrainDataset(transform=dataset.train_transform_v2)

In [ ]:
sample = train_data[123]

image = sample['image']
kps = sample['kps']

print('image shape =', image.shape)
print('key points shape =', kps.shape)

## 定义模型

我们使用的模型来自论文: [关键点检测论文](https://arxiv.org/pdf/1710.00977.pdf).

我已经把模型给实现好了, 我们需要做的仅仅是实例化这个模型.

In [ ]:
model = fk_model.FacialKpModel()
print(model)

## 在测试集上作用未训练的模型

我们的模型此时还没有得到训练, 它的表现会很差. 但是我们还是演示它在测试集上的表现, 来查看没有训练的模型会做出怎样的预测.

随后在进行训练后, 我们会再次进行这样的演示, 你可以简单对比一下训练前后的区别.


### 读取测试集, 经过forawrd输出结果

我们在测试集的一个batch上作用未训练的模型, 查看输出

In [ ]:
test_data = dataset.TestDataset(transform=dataset.train_transform_v2)

batch_size = 10
test_data_loader = dataset.get_loader(test_data, batch_size=batch_size)

In [ ]:
test_images, test_outputs, test_kps = dataset.take(model, test_data_loader)

print('test images size =', test_images.size())
print('test outputs size =', test_outputs.size())
print('key points size =', test_kps.size())

### 将预测的关键点可视化

注意经过预处理的图像是不能直接显示的, 它经过归一化变换和维度变换才能正常显示.

我在dataset中提供了一个函数`show_test`, 这个函数把所有这些过程都封装好了.

In [ ]:
dataset.show_test(test_images, test_outputs, test_kps, batch_size=3)

粉色的点是模型预测的关键点, 我们发现它们完全不能匹配人脸, 不要担心, 下面我们对模型进行训练, 训练之后再来看看情况如何.

## 训练模型

下面的代码块会对模型进行训练, 如果CUDA可用, 会使用CUDA进行训练.

In [ ]:
train_loader = dataset.get_loader(train_data, batch_size=10)

import facial_keypoints_train as train

epochs = 25

train.train(model, train_loader, epochs)

## 保存训练好的模型

恭喜, 你已经训练好了一个人脸关键点提取模型, 下面先保存这个模型的参数.

In [ ]:
train.save_model(model)

## 观察模型的表现

下面我们将训练好的模型作用于测试数据上, 来查看它的表现.

In [ ]:
# 加载已经训练好的模型
model = train.load_model()

model.eval()

# 加载测试数据集, 显示提取的关键点
test_loader = dataset.get_loader(dataset.TestDataset(transform=dataset.train_transform_v2), batch_size=20)
test_images, test_outputs, test_kps = dataset.take(model, test_loader)
dataset.show_test(test_images, test_outputs, None, batch_size=3)

## 查看卷积内核的效果



In [ ]:
import cv2
import numpy as np

weights = model.conv1.weight.data

w = weights.numpy()


image = test_data[10]['image'].data.numpy()

filtered_image = cv2.filter2D(image, -1, w[0][0])

plt.imshow(np.squeeze(filtered_image), cmap='gray')